In [ ]:
# %%
# Laboratorium 2: Filtracja obrazów RGB
# Temat: Eksperymenty z filtrami przetwarzania obrazów w przestrzeni kolorów

import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim

# %%
# 1. Wczytanie obrazu w kolorze (BGR -> RGB)
image_bgr = cv2.imread("3.pgm", cv2.IMREAD_COLOR)
image = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(5,5))
plt.imshow(image)
plt.title("Oryginalny obraz (RGB)")
plt.axis("off")
plt.show()

# %%
# 2. Implementacja filtrów

# Filtr dolnoprzepustowy - uśredniający
#(3x3) maska
avg_kernel_size = 5
filtered_avg = cv2.blur(image, (avg_kernel_size, avg_kernel_size))

# Filtr górnoprzepustowy - Laplace'a
filtered_laplace = cv2.Laplacian(image, cv2.CV_64F)
filtered_laplace = cv2.convertScaleAbs(filtered_laplace)

# Filtr medianowy
median_kernel_size = 5
filtered_median = cv2.medianBlur(image, median_kernel_size)

# %%
# 3. Wizualizacja wyników
fig, axes = plt.subplots(1, 4, figsize=(15,5))

axes[0].imshow(image)
axes[0].set_title("Oryginalny")
axes[0].axis("off")

axes[1].imshow(filtered_avg)
axes[1].set_title("Filtr uśredniający")
axes[1].axis("off")

axes[2].imshow(filtered_laplace)
axes[2].set_title("Filtr Laplace'a")
axes[2].axis("off")

axes[3].imshow(filtered_median)
axes[3].set_title("Filtr medianowy")
axes[3].axis("off")

plt.tight_layout()
plt.show()

# %%
# 4. Ocena wpływu filtrów - metryki jakości

def evaluate_filters(original, filtered):
    psnr_val = psnr(original, filtered, data_range=255)
    ssim_val = ssim(original, filtered, channel_axis=2, data_range=255)
    return psnr_val, ssim_val

metrics = {
    "Filtr uśredniający": evaluate_filters(image, filtered_avg),
    "Filtr Laplace'a": evaluate_filters(image, filtered_laplace),
    "Filtr medianowy": evaluate_filters(image, filtered_median),
}

for name, (psnr_val, ssim_val) in metrics.items():
    print(f"{name}: PSNR={psnr_val:.2f} dB, SSIM={ssim_val:.4f}")

# %%
# 5. Eksperymenty z parametrami - różne rozmiary masek
kernel_sizes = [3, 5, 7, 9]

fig, axes = plt.subplots(2, len(kernel_sizes), figsize=(15,6))

for i, k in enumerate(kernel_sizes):
    avg = cv2.blur(image, (k, k))
    med = cv2.medianBlur(image, k)

    axes[0, i].imshow(avg)
    axes[0, i].set_title(f"Uśredniający {k}x{k}")
    axes[0, i].axis("off")

    axes[1, i].imshow(med)
    axes[1, i].set_title(f"Medianowy {k}x{k}")
    axes[1, i].axis("off")

plt.tight_layout()
plt.show()

# %%
# 6. Histogramy kanałów R, G, B

def plot_rgb_histograms(image, title):
    colors = ('r', 'g', 'b')
    plt.figure(figsize=(6,4))
    for i, col in enumerate(colors):
        hist = cv2.calcHist([image], [i], None, [256], [0,256])
        plt.plot(hist, color=col)
        plt.xlim([0,256])
    plt.title(f"Histogram RGB - {title}")
    plt.xlabel("Wartości intensywności")
    plt.ylabel("Liczba pikseli")
    plt.show()

plot_rgb_histograms(image, "Oryginalny")
plot_rgb_histograms(filtered_avg, "Filtr uśredniający")
plot_rgb_histograms(filtered_laplace, "Filtr Laplace'a")
plot_rgb_histograms(filtered_median, "Filtr medianowy")
